In [1]:
import numpy as np
import random
import cv2
import os
import glob
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense
from tensorflow.keras.layers import MaxPooling2D, Activation, Flatten, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
## initial parameters
lr = 1e-2
batch_size = 200 
epochs = 15
img_dims = (96,96,3) #specifying image dimensions

data = []
labels = []

# loading image files
image_files = [f for f in glob.glob(r'C:/Users/lenovo/Desktop/Python projects/Mtech Final Project/Mtech main/dataset' + "/**/*",
recursive=True) if not os.path.isdir(f)]

random.shuffle(image_files)

In [3]:
## converting images to arrays 
for img in image_files:

    image = cv2.imread(img)
    image = cv2.resize(image, (img_dims[0],img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    ## labelling the categories
    label = img.split(os.path.sep)[-2] 
    if label == "woman":
        label = 1
    else:
        label = 0
        
    labels.append([label])  # [[1], [0], [0], ...]

In [4]:
## pre-processing
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

## spliting the dataset for training and validation
x_train,x_test,y_train,y_test = train_test_split(data, labels,
test_size=0.2,random_state=42)

## converting into categorical labels
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [5]:
# augmenting the dataset 
aug = ImageDataGenerator(rotation_range=25, 
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.2, 
                         zoom_range=0.2,
                         horizontal_flip=True, 
                         fill_mode="nearest")

In [6]:
## Defining the  Convolutional Model

## defining input shape
width = img_dims[0]
height = img_dims[1]
depth = img_dims[2]
inputShape = (height, width, depth)
dim = -1

# model creation

model = Sequential()


#Customized filters 2 stack of 2x2x1 filter.
model.add(Conv2D(32, (2,2), padding="same",input_shape=inputShape))
model.add(Conv2D(32, (2,2), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (2,2), padding="same"))
model.add(Conv2D(64, (2,2), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))

model.add(Conv2D(64, (2,2), padding="same"))
model.add(Conv2D(64, (2,2), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (2,2), padding="same"))
model.add(Conv2D(128, (2,2), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))

model.add(Conv2D(256, (2,2), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=dim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))



model.add(Flatten())

#model.add(Dense(1024))
#model.add(Activation("relu"))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation("sigmoid"))

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

In [9]:
## compile the model
opt= keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

## fit the model
h = model.fit_generator(aug.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test,y_test),
                        steps_per_epoch=len(x_train) // batch_size,
                        epochs=epochs, verbose=1)

## save the model. This will save your model so that you can use this model again when needed without training it again.
model.save('test_gender_predictor_2x2_with_15_epochs.model')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
17/17 [==============================] - 257s 15s/step - loss: 6.4341 - accuracy: 0.6169 - val_loss: 3758.8540 - val_accuracy: 0.4800
Epoch 2/15
17/17 [==============================] - 272s 16s/step - loss: 2.0701 - accuracy: 0.7096 - val_loss: 412.8250 - val_accuracy: 0.4948
Epoch 3/15
17/17 [==============================] - 269s 16s/step - loss: 1.3473 - accuracy: 0.7245 - val_loss: 11.3394 - val_accuracy: 0.7228
Epoch 4/15
17/17 [==============================] - 270s 16s/step - loss: 1.2056 - accuracy: 0.7610 - val_loss: 40.4905 - val_accuracy: 0.6380
Epoch 5/15
17/17 [==============================] - 269s 16s/step - loss: 1.1270 - accuracy: 0.7571 - val_loss: 39.0428 - val_accuracy: 0.4903
Epoch 6/15
17/17 [==============================] - 146s 9s/step - loss: 1.1747 - accuracy: 0.7656 - val_loss: 12.2244 - val_accuracy: 0.4948
Epoch 7/15
17/17 [==============================] - 143s 8s/step